# LGBM - 전체 데이터셋

In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lg-aimers/sample_submission.csv
/kaggle/input/lg-aimers/train.csv
/kaggle/input/lg-aimers/test.csv


In [18]:
data_path = '/kaggle/input/lg-aimers/'

train = pd.read_csv(data_path + 'train.csv', index_col= 'ID')
test = pd.read_csv(data_path + 'test.csv', index_col= 'ID')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col= 'ID')

## 1. 데이터 통합

In [19]:
all_data = pd.concat([train, test], ignore_index= True)
all_data = all_data.drop('임신 성공 여부', axis= 1)

In [20]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [21]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    all_data[col] = all_data[col].astype(str)

## 2. 피처 엔지니어링: 원 핫 인코딩

In [22]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(all_data[categorical_columns])

encoded_cat_matrix

<346418x205 sparse matrix of type '<class 'numpy.float64'>'
	with 16281646 stored elements in Compressed Sparse Row format>

In [23]:
all_columns = all_data.columns
all_columns

Index(['시술 시기 코드', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '특정 시술 유형',
       '배란 자극 여부', '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부',
       '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
       '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환',
       '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제',
       '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
       '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유', '총 시술 횟수',
       '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수',
       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수',
       '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수',
       '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수',
       '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처',
       '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부',
       '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자

In [24]:
remaining_columns = list(set(all_columns) - set(categorical_columns))

In [25]:
from scipy import sparse

extracted_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_columns]),
                               encoded_cat_matrix],
                              format='csr')

## 3. 데이터 나누기

In [26]:
num_train = len(train) 

X = extracted_data_sprs[:num_train]
X_test = extracted_data_sprs[num_train:]

y = train['임신 성공 여부'].values

## 4. 모델 훈련

### 4-1. 층화 K 폴드

In [27]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

### 4-2. 베이지안 최적화

In [28]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      random_state=0)

# 베이지안 최적화용 데이터셋
bayes_dtrain = lgb.Dataset(X_train, y_train)
bayes_dvalid = lgb.Dataset(X_valid, y_valid)

In [29]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
# TODO: 점점 줄여나가자!!!!
param_bounds = {
    'num_leaves': (20, 100),  
    'lambda_l1': (0, 2),  
    'lambda_l2': (0, 2),  
    'feature_fraction': (0.5, 1),  
    'bagging_fraction': (0.5, 1),  
    'min_child_samples': (5, 50),  
    'min_child_weight': (1, 50)  
}

# 값이 고정된 하이퍼파라미터
fixed_params = {'objective': 'binary', # binary classification
                'learning_rate': 0.005, # 0.01~0.001
                'bagging_freq': 1, # 0 or 1
                'force_row_wise': True,
                'random_state': 1991}

In [30]:
from sklearn.metrics import roc_auc_score
from lightgbm import early_stopping

def eval_function(num_leaves, lambda_l1, lambda_l2, feature_fraction,
                  bagging_fraction, min_child_samples, min_child_weight):
    '''최적화하려는 평가지표 계산 함수'''
    
    # 베이지안 최적화를 수행할 하이퍼파라미터 
    params = {'num_leaves': int(round(num_leaves)),
              'lambda_l1': lambda_l1,
              'lambda_l2': lambda_l2,
              'feature_fraction': feature_fraction,
              'bagging_fraction': bagging_fraction,
              'min_child_samples': int(round(min_child_samples)),
              'min_child_weight': min_child_weight,
              'feature_pre_filter': False}
    # 고정된 하이퍼파라미터도 추가
    params.update(fixed_params)
    
    print('하이퍼파라미터:', params)    
    
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=params, 
                           train_set=bayes_dtrain,
                           num_boost_round=2500,
                           valid_sets=bayes_dvalid,
                           callbacks=[early_stopping(stopping_rounds=100)])
    # 검증 데이터로 예측 수행
    preds = lgb_model.predict(X_valid) 
    # roc-auc 계산
    roc_auc = roc_auc_score(y_valid, preds)
    print(f'roc-auc : {roc_auc}\n')
    
    return roc_auc

In [31]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function,      # 평가지표 계산 함수
                                 pbounds=param_bounds, # 하이퍼파라미터 범위
                                 random_state=0)

In [32]:
# 베이지안 최적화 수행
# TODO: init_points 10~15, n_iter 30~70
optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
하이퍼파라미터: {'num_leaves': 55, 'lambda_l1': 1.2055267521432877, 'lambda_l2': 1.0897663659937937, 'feature_fraction': 0.8575946831862098, 'bagging_fraction': 0.7744067519636624, 'min_child_samples': 24, 'min_child_weight': 32.64881154026615, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 52867, number of negative: 152213
[LightGBM] [Info] Total Bins 951
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 221
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.257787 -> initscore=-1.057502
[LightGBM] [Info] Start training from score -1.057502
Training until validation scores don't improve for 100 rounds
Early

In [33]:
# 평가함수 점수가 최대일 때 하이퍼파라미터
max_params = optimizer.max['params']
max_params

{'bagging_fraction': 0.5024417571224058,
 'feature_fraction': 0.8828297423537881,
 'lambda_l1': 0.12365931116742002,
 'lambda_l2': 1.8847998821750824,
 'min_child_samples': 6.346129551978789,
 'min_child_weight': 33.74412877738659,
 'num_leaves': 32.22730568024136}

In [34]:
# 정수형 하이퍼파라미터 변환
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))

In [35]:
# 값이 고정된 하이퍼파라미터 추가
max_params.update(fixed_params)

In [36]:
max_params

{'bagging_fraction': 0.5024417571224058,
 'feature_fraction': 0.8828297423537881,
 'lambda_l1': 0.12365931116742002,
 'lambda_l2': 1.8847998821750824,
 'min_child_samples': 6,
 'min_child_weight': 33.74412877738659,
 'num_leaves': 32,
 'objective': 'binary',
 'learning_rate': 0.005,
 'bagging_freq': 1,
 'force_row_wise': True,
 'random_state': 1991}

In [41]:
def lgb_roc_auc(y_pred, dataset):
    y_true = dataset.get_label()
    return "roc_auc", roc_auc_score(y_true, y_pred), True  # (지표 이름, 값, 높은 값이 더 좋은지 여부)

In [42]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0]) 

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGBM 전용 검증 데이터셋
                          
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=max_params,    # 최적 하이퍼파라미터
                          train_set=dtrain,     # 훈련 데이터셋
                          num_boost_round=2500, # 부스팅 반복 횟수
                          valid_sets=dvalid,    # 성능 평가용 검증 데이터셋
                          feval = lgb_roc_auc,
                          callbacks=[early_stopping(stopping_rounds=100)])
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측 
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 ROC-AUC
    roc_auc = roc_auc_score(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} roc-auc : {roc_auc}\n')

######################################## 폴드 1 / 폴드 5 ########################################
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Total Bins 904
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 198
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1889]	valid_0's binary_logloss: 0.487434	valid_0's roc_auc: 0.740021
폴드 1 roc-auc : 0.7400205130587598

######################################## 폴드 2 / 폴드 5 ########################################
[LightGBM] [Info] Number of positive: 52983, number of negative: 152098
[LightGBM] [Info] Total Bins 913
[LightGBM] [Info] Number of data points in the train set: 205081, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258352 -> initscore=-1.054

## 5. 최종 결과

In [43]:
print('OOF 검증 데이터 roc-auc :', roc_auc_score(y, oof_val_preds))

OOF 검증 데이터 roc-auc : 0.7402237231097757


In [44]:
submission['probability'] = oof_test_preds
submission.to_csv('lgbm_submission.csv')

In [46]:
submission = pd.read_csv('/kaggle/working/lgbm_submission.csv')
submission.head()

,ID,probability,임신 성공 여부
0,TEST_00000,0.001947,0.001902
1,TEST_00001,0.001889,0.001842
2,TEST_00002,0.155794,0.155787
3,TEST_00003,0.104613,0.104139
4,TEST_00004,0.498481,0.497891


In [50]:
submission = submission.drop('임신 성공 여부', axis=1)

In [51]:
submission.to_csv('lgbm_submission.csv')
submission.head()

,ID,probability
0,TEST_00000,0.001947
1,TEST_00001,0.001889
2,TEST_00002,0.155794
3,TEST_00003,0.104613
4,TEST_00004,0.498481
